In [63]:
import numpy as np
from numpy.linalg import inv
from scipy.special import expit
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy import stats
import datasets

In [65]:
X, y = datasets.htwt()
N, D = X.shape
Xaug = np.hstack([np.ones((N, 1)), X])

In [66]:
model = sm.GLM(y, Xaug, family = sm.families.Binomial()).fit()
np.mean((model.predict(Xaug) > 0.5) != y)

0.11904761904761904

In [108]:
# probit regression
Psi = stats.norm.cdf
psi = stats.norm.pdf

y_tilde = 2 * y - 1

loss = lambda w: -Xaug.dot(w).dot(y_tilde)
w = minimize(fun, np.zeros(D + 1)).x

def grad(w):
    s = 0
    eta = Xaug.dot(w)
    for i in xrange(N):
        s += y_tilde[i] * Xaug[i].dot(psi(eta[i])) / Psi(y_tilde[i] * eta[i])
    return s
        
print grad(w)

def grad(w):
    s = 0
    eta = Xaug.dot(w)
    yeta = y_tilde * eta
    psieta = psi(eta)
    zzz = Xaug.T.dot(psieta)
    print zzz.shape
    for i in xrange(N):
        s += y_tilde[i] * Xaug[i].dot(psieta[i]) / Psi(yeta[i])
    return s
        
print grad(w)


# w = minimize(fun, np.zeros(D + 1), jac = grad, hess = hess).x
np.mean((Xaug.dot(w) > 0) != y)

[   3.78183395  193.42521433 -138.56560623]
(3,)
[   3.78183395  193.42521433 -138.56560623]


0.11904761904761904

In [10]:
# iteratively reweighted least squares -- murphy p. 253
w = np.zeros(D + 1)

for _ in xrange(5):
    eta = Xaug.dot(w)
    mu = expit(eta)
    S = np.diag(mu * (1 - mu))
    z = eta + inv(S).dot(y - mu)
    w = inv(Xaug.T.dot(S).dot(Xaug)).dot(Xaug.T).dot(S).dot(z)

np.mean((expit(Xaug.dot(w)) > 0.5) != y)

0.11904761904761904

In [8]:
def fun(w):
    eta = Xaug.dot(w)
    A = np.log(1 + np.exp(eta))
    return -sum(eta * y - A)
        
w = minimize(fun, np.zeros(D + 1)).x

np.mean((expit(Xaug.dot(w)) > 0.5) != y)

0.11904761904761904